In [2]:
#Bibliotecas para poder trabajar con Spark
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf spark-3.5.0-bin-hadoop3.tgz
#Configuración de Spark con Python
!pip install -q findspark
!pip install pyspark

#Estableciendo variable de entorno
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

#Buscando e inicializando la instalación de Spark
import findspark
findspark.init()
findspark.find()

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [589 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:5 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,012 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,185 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,389 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:12 http://archive.ubuntu.com/ubuntu jammy InRe

'/content/spark-3.5.0-bin-hadoop3'

In [67]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression

In [4]:
spark_session = SparkSession.builder.appName('Entregable1').getOrCreate()
spark_session

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [45]:
df = spark_session.read.csv('/content/drive/My Drive/Colab Notebooks/bank.csv', inferSchema=True, header=True)

In [46]:
df.show()

+---+-----------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
|age|        job| marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|deposit|
+---+-----------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
| 59|     admin.| married|secondary|     no|   2343|    yes|  no|unknown|  5|  may|    1042|       1|   -1|       0| unknown|    yes|
| 56|     admin.| married|secondary|     no|     45|     no|  no|unknown|  5|  may|    1467|       1|   -1|       0| unknown|    yes|
| 41| technician| married|secondary|     no|   1270|    yes|  no|unknown|  5|  may|    1389|       1|   -1|       0| unknown|    yes|
| 55|   services| married|secondary|     no|   2476|    yes|  no|unknown|  5|  may|     579|       1|   -1|       0| unknown|    yes|
| 54|     admin.| married| tertiary|     no|    184|     no|  

In [47]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: integer (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- deposit: string (nullable = true)



In [48]:
data = df.drop(*['contact', 'day', 'month', 'default'])
data.columns

['age',
 'job',
 'marital',
 'education',
 'balance',
 'housing',
 'loan',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'deposit']

In [49]:
data.describe().show()

+-------+------------------+-------+--------+---------+------------------+-------+-----+------------------+------------------+------------------+------------------+--------+-------+
|summary|               age|    job| marital|education|           balance|housing| loan|          duration|          campaign|             pdays|          previous|poutcome|deposit|
+-------+------------------+-------+--------+---------+------------------+-------+-----+------------------+------------------+------------------+------------------+--------+-------+
|  count|             11162|  11162|   11162|    11162|             11162|  11162|11162|             11162|             11162|             11162|             11162|   11162|  11162|
|   mean|41.231947679627304|   NULL|    NULL|     NULL|1528.5385235620856|   NULL| NULL|371.99381831213043| 2.508421429851281| 51.33040673714388|0.8325568894463358|    NULL|   NULL|
| stddev|11.913369192215518|   NULL|    NULL|     NULL| 3225.413325946149|   NULL| NULL|34

In [50]:
# Para contar cuantos null hay en cada columna
data_agg = data.agg(*[f.count(f.when(f.isnull(c), c)).alias(c) for c in data.columns])
data_agg.show()

+---+---+-------+---------+-------+-------+----+--------+--------+-----+--------+--------+-------+
|age|job|marital|education|balance|housing|loan|duration|campaign|pdays|previous|poutcome|deposit|
+---+---+-------+---------+-------+-------+----+--------+--------+-----+--------+--------+-------+
|  0|  0|      0|        0|      0|      0|   0|       0|       0|    0|       0|       0|      0|
+---+---+-------+---------+-------+-------+----+--------+--------+-----+--------+--------+-------+



In [51]:
# value counts of columns
data.groupBy('job').count().show()
print()
data.groupBy('marital').count().show()
print()
data.groupBy('education').count().show()
print()
data.groupBy('loan').count().show()
print()
data.groupBy('poutcome').count().show()
print()
data.groupBy('deposit').count().show()

+-------------+-----+
|          job|count|
+-------------+-----+
|   management| 2566|
|      retired|  778|
|      unknown|   70|
|self-employed|  405|
|      student|  360|
|  blue-collar| 1944|
| entrepreneur|  328|
|       admin.| 1334|
|   technician| 1823|
|     services|  923|
|    housemaid|  274|
|   unemployed|  357|
+-------------+-----+


+--------+-----+
| marital|count|
+--------+-----+
|divorced| 1293|
| married| 6351|
|  single| 3518|
+--------+-----+


+---------+-----+
|education|count|
+---------+-----+
|  unknown|  497|
| tertiary| 3689|
|secondary| 5476|
|  primary| 1500|
+---------+-----+


+----+-----+
|loan|count|
+----+-----+
|  no| 9702|
| yes| 1460|
+----+-----+


+--------+-----+
|poutcome|count|
+--------+-----+
| success| 1071|
| unknown| 8326|
|   other|  537|
| failure| 1228|
+--------+-----+


+-------+-----+
|deposit|count|
+-------+-----+
|     no| 5873|
|    yes| 5289|
+-------+-----+



In [52]:
data.dtypes

[('age', 'int'),
 ('job', 'string'),
 ('marital', 'string'),
 ('education', 'string'),
 ('balance', 'int'),
 ('housing', 'string'),
 ('loan', 'string'),
 ('duration', 'int'),
 ('campaign', 'int'),
 ('pdays', 'int'),
 ('previous', 'int'),
 ('poutcome', 'string'),
 ('deposit', 'string')]

In [53]:
# Crear objetos StringIndexer para cada columna categórica
indexer_job = StringIndexer(inputCol='job', outputCol='job_Index')
indexer_marital = StringIndexer(inputCol='marital',outputCol='marital_Index')
indexer_education = StringIndexer(inputCol='education',outputCol='education_Index')
indexer_housing = StringIndexer(inputCol='housing',outputCol='housing_Index')
indexer_loan = StringIndexer(inputCol='loan',outputCol='loan_Index')
indexer_poutcome = StringIndexer(inputCol='poutcome',outputCol='poutcome_Index')
indexer_deposit = StringIndexer(inputCol='deposit',outputCol='deposit_Index')

# Transformar los datos aplicando la codificación a cada columna
data = indexer_job.fit(data).transform(data)
data = indexer_marital.fit(data).transform(data)
data = indexer_education.fit(data).transform(data)
data = indexer_housing.fit(data).transform(data)
data = indexer_loan.fit(data).transform(data)
data = indexer_poutcome.fit(data).transform(data)
data = indexer_deposit.fit(data).transform(data)

In [54]:
data.select('*').show()

+---+-----------+--------+---------+-------+-------+----+--------+--------+-----+--------+--------+-------+---------+-------------+---------------+-------------+----------+--------------+-------------+
|age|        job| marital|education|balance|housing|loan|duration|campaign|pdays|previous|poutcome|deposit|job_Index|marital_Index|education_Index|housing_Index|loan_Index|poutcome_Index|deposit_Index|
+---+-----------+--------+---------+-------+-------+----+--------+--------+-----+--------+--------+-------+---------+-------------+---------------+-------------+----------+--------------+-------------+
| 59|     admin.| married|secondary|   2343|    yes|  no|    1042|       1|   -1|       0| unknown|    yes|      3.0|          0.0|            0.0|          1.0|       0.0|           0.0|          1.0|
| 56|     admin.| married|secondary|     45|     no|  no|    1467|       1|   -1|       0| unknown|    yes|      3.0|          0.0|            0.0|          0.0|       0.0|           0.0|     

In [55]:
one_hot_enconder = OneHotEncoder(
    inputCols=['job_Index','marital_Index','education_Index','housing_Index',
               'loan_Index','poutcome_Index','deposit_Index'],
    outputCols=['job_onehot','martial_onehot','education_onehot',
                'housing_onehot','loan_onehot','poutcome_onehot','deposit_onehot'])

data = one_hot_enconder.fit(data).transform(data)

data.select('job', 'job_Index', 'job_onehot','education','education_Index','education_onehot').show(10)

+----------+---------+--------------+---------+---------------+----------------+
|       job|job_Index|    job_onehot|education|education_Index|education_onehot|
+----------+---------+--------------+---------+---------------+----------------+
|    admin.|      3.0|(11,[3],[1.0])|secondary|            0.0|   (3,[0],[1.0])|
|    admin.|      3.0|(11,[3],[1.0])|secondary|            0.0|   (3,[0],[1.0])|
|technician|      2.0|(11,[2],[1.0])|secondary|            0.0|   (3,[0],[1.0])|
|  services|      4.0|(11,[4],[1.0])|secondary|            0.0|   (3,[0],[1.0])|
|    admin.|      3.0|(11,[3],[1.0])| tertiary|            1.0|   (3,[1],[1.0])|
|management|      0.0|(11,[0],[1.0])| tertiary|            1.0|   (3,[1],[1.0])|
|management|      0.0|(11,[0],[1.0])| tertiary|            1.0|   (3,[1],[1.0])|
|   retired|      5.0|(11,[5],[1.0])|secondary|            0.0|   (3,[0],[1.0])|
|technician|      2.0|(11,[2],[1.0])|secondary|            0.0|   (3,[0],[1.0])|
|  services|      4.0|(11,[4

In [56]:
data.show()

+---+-----------+--------+---------+-------+-------+----+--------+--------+-----+--------+--------+-------+---------+-------------+---------------+-------------+----------+--------------+-------------+--------------+--------------+----------------+--------------+-------------+---------------+--------------+
|age|        job| marital|education|balance|housing|loan|duration|campaign|pdays|previous|poutcome|deposit|job_Index|marital_Index|education_Index|housing_Index|loan_Index|poutcome_Index|deposit_Index|    job_onehot|martial_onehot|education_onehot|housing_onehot|  loan_onehot|poutcome_onehot|deposit_onehot|
+---+-----------+--------+---------+-------+-------+----+--------+--------+-----+--------+--------+-------+---------+-------------+---------------+-------------+----------+--------------+-------------+--------------+--------------+----------------+--------------+-------------+---------------+--------------+
| 59|     admin.| married|secondary|   2343|    yes|  no|    1042|       

In [62]:

assembler = VectorAssembler(inputCols=['age',
                                       'job_Index',
                                       'marital_Index',
                                       'education_Index',
                                       'balance',
                                       'housing_Index',
                                       'loan_Index',
                                       'duration',
                                       'campaign',
                                       'pdays',
                                       'previous',
                                       'poutcome_Index',
                                       'job_onehot',
                                       'martial_onehot',
                                       'housing_onehot',
                                       'education_onehot',
                                       'loan_onehot',
                                       'poutcome_onehot'],
                           outputCol='features')

In [63]:
final_data = assembler.transform(data)

In [64]:
final_data.select('features','deposit_Index').show()

+--------------------+-------------+
|            features|deposit_Index|
+--------------------+-------------+
|(33,[0,1,4,5,7,8,...|          1.0|
|(33,[0,1,4,7,8,9,...|          1.0|
|(33,[0,1,4,5,7,8,...|          1.0|
|(33,[0,1,4,5,7,8,...|          1.0|
|(33,[0,1,3,4,7,8,...|          1.0|
|(33,[0,2,3,5,6,7,...|          1.0|
|(33,[0,3,4,5,6,7,...|          1.0|
|(33,[0,1,2,4,5,7,...|          1.0|
|(33,[0,1,4,5,7,8,...|          1.0|
|(33,[0,1,2,4,5,7,...|          1.0|
|(33,[0,1,2,4,5,7,...|          1.0|
|(33,[0,1,4,5,7,8,...|          1.0|
|(33,[0,3,4,5,6,7,...|          1.0|
|(33,[0,1,2,3,4,5,...|          1.0|
|(33,[0,1,2,3,4,5,...|          1.0|
|(33,[0,2,3,4,5,7,...|          1.0|
|(33,[0,1,2,3,4,5,...|          1.0|
|(33,[0,1,4,5,7,8,...|          1.0|
|(33,[0,1,4,5,7,8,...|          1.0|
|(33,[0,1,2,4,5,6,...|          1.0|
+--------------------+-------------+
only showing top 20 rows



In [65]:
model_df = final_data.select(['features','deposit_Index'])
model_df = model_df.withColumnRenamed("deposit_Index","label")
model_df.printSchema()

root
 |-- features: vector (nullable = true)
 |-- label: double (nullable = false)



In [66]:
training_df, test_df = model_df.randomSplit([0.75,0.25])

In [68]:
model = LogisticRegression().fit(training_df)
model_summary = model.summary
print(f'ACCURACY: {model_summary.accuracy}')
print(f'AREA UNDER ROC: {model_summary.areaUnderROC}')

ACCURACY: 0.7995468638206534
AREA UNDER ROC: 0.8806670492647813


In [73]:
#Precision of both classes
print(model_summary.precisionByLabel)

[0.7880135535789919, 0.8144104803493449]


In [74]:
#Recall of both classes
print(model_summary.recallByLabel)

[0.8454896614405817, 0.7488080301129235]


In [77]:
#Get Predictions
predictions = model.transform(test_df)
predictions.select('label','prediction').show(50)

+-----+----------+
|label|prediction|
+-----+----------+
|  0.0|       0.0|
|  1.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  1.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       0.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  0.0|       1.0|
|  0.0|       1.0|
|  1.0|       0.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  0.0|     